In [8]:
import os
from pathlib import Path
import re
import time

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

In [11]:
# Konfiguration
input_dir = Path('data/raw')
text_dir = Path('data/text')
table_dir = Path('data/table')
tabletext_dir = Path('data/tabletext')
output_dir = Path('data/processed')
min_length = 100

In [4]:
# Alle .md-Dateien im Eingabeordner durchgehen
for input_file in input_dir.glob('*.md'):
    # Datei einlesen
    with input_file.open('r', encoding='utf-8') as f:
        lines = f.readlines()

    # Zeilen filtern
    filtered_lines = []
    for line in lines:
        stripped = line.strip()
        if len(stripped) >= min_length:
            filtered_lines.append(line)

    # Neuen Dateinamen erstellen
    output_filename = input_file.stem + '_text.md'
    output_file = text_dir / output_filename

    # Gefilterte Zeilen schreiben
    with output_file.open('w', encoding='utf-8') as f:
        f.writelines(filtered_lines)

In [5]:
# Alle .md-Dateien im Eingabeordner durchgehen
for input_file in input_dir.glob('*.md'):
    # Datei einlesen
    with input_file.open('r', encoding='utf-8') as f:
        lines = f.readlines()

    # Zeilen filtern: nur Tabellenbereiche behalten
    filtered_lines = []
    inside_table = False
    for line in lines:
        stripped = line.strip()
        if stripped == "--- Tabelle Start ---":
            inside_table = True
            filtered_lines.append(line)
        elif stripped == "--- Tabelle Ende ---":
            inside_table = False
            filtered_lines.append(line)
        elif inside_table:
            filtered_lines.append(line)
        # Alles außerhalb der Tabellenbereiche wird ignoriert

    # Zweiter Durchgang: Tabellen nummerieren und Endmarkierungen entfernen
    final_lines = []
    table_count = 0
    for line in filtered_lines:
        stripped = line.strip()
        if stripped == "--- Tabelle Start ---":
            table_count += 1
            final_lines.append(f"{table_count}. Tabelle\n")
        elif stripped == "--- Tabelle Ende ---":
            continue  # Zeile wird entfernt
        else:
            final_lines.append(line)

    # Neuen Dateinamen erstellen
    output_filename = input_file.stem + '_tabelle.md'
    output_file = table_dir / output_filename

    # Gefilterte und umgewandelte Zeilen schreiben
    with output_file.open('w', encoding='utf-8') as f:
        f.writelines(final_lines)

In [6]:
folder_path = table_dir  # Dein Ordnerpfad
table_data = {}          # Zentrales Dictionary für alle Firmen

def extract_firm_name(filename):
    # Extrahiert alles vor dem ersten Unterstrich und wandelt in Kleinbuchstaben um
    return filename.split('_')[0].lower()

for filename in os.listdir(folder_path):
    if filename.endswith(".md"):
        filepath = os.path.join(folder_path, filename)

        tables = {}
        current_table_number = None
        current_table_lines = []

        with open(filepath, "r", encoding="utf-8") as file:
            for line in file:
                match = re.match(r"(\d+)\. Tabelle", line.strip())
                if match:
                    if current_table_number is not None and current_table_lines:
                        tables[current_table_number] = ''.join(current_table_lines).strip()
                    current_table_number = int(match.group(1))
                    current_table_lines = []
                elif current_table_number is not None:
                    current_table_lines.append(line)

            # Letzte Tabelle hinzufügen
            if current_table_number is not None and current_table_lines:
                tables[current_table_number] = ''.join(current_table_lines).strip()

        # Firmenname extrahieren
        firm_name = extract_firm_name(filename)

        # Direkt auf erster Ebene speichern: table_data['bmw'][1]
        table_data[firm_name] = tables


----------



# Test

In [25]:
table = table_data['daimler'][50]
print(table)

| Absatz? | 447.790 Fahrzeuge | leicht unter Vorjahresniveau |
| --- | --- | --- |
| Anteil elektrifizierter Fahrzeuge (xEV)? | 5% | 6-8% |
| Umsatz | 20.288 Mio.€ | auf Vorjahresniveau |
| Bereinigte Umsatzrendite | 15,1% | 12-14% |
| Bereinigte Cash Conversion Rate | 1,0 | 0,6-0,8 |
| Sachinvestitionent | 351 Mio.€ | deutlich iiber Vorjahresniveau |
| Forschungs- und Entwicklungsleistungen! | 873 Mio.€ | deutlich iiber Vorjahresniveau |
| Mercedes-Benz Mobility | | |
| Neugeschaft! | 62.014 Mio.€ | leicht iiber Vorjahresniveau |
| Vertragsvolumen | 135.027 Mio.€ | auf Vorjahresniveau |
| Umsatz | 26.718 Mio. € | auf Vorjahresniveau |
| Bereinigte Eigenkapitalrendite! | 12,3% | 10-12% |


In [26]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=1,max_tokens=None,google_api_key=os.getenv("GOOGLE_API_KEY"))
prompt = f"""Erstelle zu der untenstehenden Tabelle einen Fließtext, der den Inhalt der Tabelle zusammenfasst.
        Verfasse zu jeder Dateneinheit einen eigenen, klar verständlichen Satz.
        Jeder Satz soll eigenständig sinnvoll sein, damit der Kontext auch dann erhalten bleibt, wenn man nur einen einzelnen Satz liest.
        Verwende kurze und präzise Formulierungen. Antworte nur mit dem Fließtext, ohne zusätzliche Erklärungen oder Einleitungen.

        Tabelle:
        {table}
        """

response = llm.invoke([HumanMessage(content=prompt)])
response.content

'Der Fahrzeugabsatz lag mit 447.790 Einheiten leicht unter dem Vorjahresniveau. Der Anteil elektrifizierter Fahrzeuge (xEV) betrug 5%, erwartet wurden 6-8%. Der Umsatz erreichte 20.288 Mio.€ und bewegte sich damit auf Vorjahresniveau. Die bereinigte Umsatzrendite lag bei 15,1%, während 12-14% erwartet wurden. Die bereinigte Cash Conversion Rate betrug 1,0, erwartet wurden 0,6-0,8. Die Sachinvestitionen beliefen sich auf 351 Mio.€ und lagen damit deutlich über dem Vorjahresniveau. Die Forschungs- und Entwicklungsleistungen erreichten 873 Mio.€ und lagen ebenfalls deutlich über dem Vorjahresniveau. Bei Mercedes-Benz Mobility betrug das Neugeschäft 62.014 Mio.€, was leicht über dem Vorjahresniveau liegt. Das Vertragsvolumen lag mit 135.027 Mio.€ auf Vorjahresniveau. Der Umsatz von Mercedes-Benz Mobility betrug 26.718 Mio. € und entsprach dem Vorjahresniveau. Die bereinigte Eigenkapitalrendite betrug 12,3%, während 10-12% erwartet wurden.'

-------------

In [9]:
load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=1,
    max_tokens=None,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

output_dir = "data/tabletext"
os.makedirs(output_dir, exist_ok=True)

for firm_name, tables in table_data.items():
    filename = f"{firm_name}_tabletext.md"
    filepath = os.path.join(output_dir, filename)

    # Wenn Datei nicht existiert, erstelle sie leer
    if not os.path.exists(filepath):
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(f"# Tabelletexte für {firm_name}\n\n")

    # Lade aktuellen Inhalt der Datei
    with open(filepath, "r", encoding="utf-8") as f:
        existing_content = f.read()

    for idx, table in tables.items():
        marker = f"Table_{idx}_text"

        # Prüfe, ob dieser Abschnitt schon existiert
        if re.search(fr"^{re.escape(marker)}\s*$", existing_content, re.MULTILINE):
            continue  # Text schon vorhanden → überspringen

        # Prompt vorbereiten
        prompt = f"""Erstelle zu der untenstehenden Tabelle einen Fließtext, der den Inhalt der Tabelle zusammenfasst.
                Verfasse zu jeder Dateneinheit einen eigenen, klar verständlichen Satz.
                Jeder Satz soll eigenständig sinnvoll sein, damit der Kontext auch dann erhalten bleibt, wenn man nur einen einzelnen Satz liest.
                Verwende kurze und präzise Formulierungen. Antworte nur mit dem Fließtext, ohne zusätzliche Erklärungen oder Einleitungen.

                Tabelle:
                {table}
                """

        # LLM anfragen
        response = llm.invoke([HumanMessage(content=prompt)])
        result_text = response.content.strip()

        # Neuen Text anhängen
        with open(filepath, "a", encoding="utf-8") as f:
            f.write(f"\n{marker}\n{result_text}\n")

        time.sleep(4.5)  # Damit du bei 15 Requests pro Minute bleibst

In [ ]:
# Pfad zum Verzeichnis mit den .md-Dateien
directory = tabletext_dir

# Regulärer Ausdruck für Zeilen wie 'table_1_text', 'table_42_text' etc.
marker_pattern = re.compile(r"^table_\d+_text\s*$", re.IGNORECASE)

# Alle .md-Dateien im Verzeichnis durchgehen
for filename in os.listdir(directory):
    if filename.endswith(".md"):
        filepath = os.path.join(directory, filename)

        with open(filepath, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # Nur Zeilen behalten, die weder leer noch ein table_*_text-Marker sind
        cleaned_lines = [
            line for line in lines
            if not marker_pattern.match(line.strip()) and line.strip() != ""
        ]

        # Datei überschreiben mit den bereinigten Zeilen
        with open(filepath, "w", encoding="utf-8") as f:
            f.writelines(cleaned_lines)

In [14]:
os.makedirs(output_dir, exist_ok=True)

# Alle Textdateien in data/text durchgehen
for filename in os.listdir(text_dir):
    if filename.endswith("_2023_text.md"):
        firm_name = filename.replace("_2023_text.md", "")
        
        # Pfade vorbereiten
        text_path = os.path.join(text_dir, filename)
        tabletext_filename = f"{firm_name}_tabletext.md"
        tabletext_path = os.path.join(tabletext_dir, tabletext_filename)
        processed_filename = f"{firm_name}_2023_processed.md"
        processed_path = os.path.join(output_dir, processed_filename)

        # Beide Dateien müssen existieren
        if not os.path.exists(tabletext_path):
            print(f"Tabelle fehlt für {firm_name}: {tabletext_filename}")
            continue

        # Inhalte einlesen
        with open(text_path, "r", encoding="utf-8") as f:
            text_content = f.read()

        with open(tabletext_path, "r", encoding="utf-8") as f:
            tabletext_content = f.read()

        # Inhalte kombinieren: erst Text, dann Tabelle
        combined_content = text_content.strip() + "\n\n" + tabletext_content.strip()

        # Neue Datei schreiben
        with open(processed_path, "w", encoding="utf-8") as f:
            f.write(combined_content)
